In [ ]:
!pip install langchain_core
!pip install langchain_openai

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

messages = [
        SystemMessage(content = "Eres un asistente personal de IA"),
        HumanMessage(content = "¿Me ayudas a preparar arroy chino?"),
        AIMessage(content = "¡Claro que si!"),
        HumanMessage(content = "Tengo los siguientes ingredientes: Arroz, raices, aceite de oliva, cebolla, pimenton, lomo de cerdo, pollo, jamon, camarones, salsa soja"),
        AIMessage("Aqui tienes la receta:")
]


In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Introduce tu clave API de Google AI: ")


In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [ ]:
messages = [
    (
        "system",
        "Eres un asistente personal de IA.",
    ),
    ("human", "¿Me ayudas a preparar arroz chino?"),
    ("ai", "¡Claro que si!"),
    ("human", "Tengo los siguientes ingredientes: Arroz, raices, aceite de oliva, cebolla, pimenton, lomo de cerdo, pollo, jamon, camarones, salsa soja"),
    ("ai", "Aqui tienes la receta:")
]
ai_msg = llm.invoke(messages)
ai_msg

print(ai_msg.content)

Encadenar el Modelo con Plantillas de Prompts
Podemos mejorar la funcionalidad del modelo encadenándolo con plantillas de prompts. Esto permite adaptar el comportamiento del modelo de forma dinámica, por ejemplo, para traducciones entre diferentes idiomas.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Crear una plantilla de prompt para traducciones
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente útil que traduce {input_language} a {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

# Encadenar la plantilla con el modelo de Google AI
chain = prompt | llm

# Invocar la cadena con parámetros específicos
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Spanish",
        "input": "I love programming.",
    }
)

Configuración de Seguridad en los Modelos Gemini
Los modelos Gemini de Google AI tienen configuraciones de seguridad por defecto que ayudan a prevenir la generación de contenido dañino. Sin embargo, estas configuraciones pueden ser ajustadas para adaptarse mejor a las necesidades de tu aplicación.

Por ejemplo, si estás recibiendo muchos "Safety Warnings" en tus respuestas, puedes modificar los umbrales de bloqueo para categorías específicas de contenido:

Este código ajusta la configuración de seguridad para permitir contenido que normalmente podría estar bloqueado por defecto.


In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

# Desactivar el bloqueo para contenido peligroso
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

#Conclusión

##Los modelos de Google ofrecen una solución robusta para construir aplicaciones de chat avanzadas, desde la configuración básica hasta la generación de respuestas y el uso de plantillas de prompts, hemos explorado cómo ajustar las configuraciones de seguridad para personalizar el comportamiento del modelo según las necesidades de tu aplicación.utilizando los modelos de Google AI con LangChain específicamente con Gemini.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
messages="¿Cuanto es 2 * 75?"

ai_msg = llm.invoke(messages)
ai_msg.content

Ahora lo entrenamos para que innfiera datos

In [ ]:
# Con estos ejemplos se entren el modelo
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [{'input':'2 ⌂ 2','output':'4'},
            {'input':'2 ⌂ 3','output':'5'}]



In [ ]:
# Se crea el template de ejemplo
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

In [ ]:
# Se une el template con los ejemplos
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [ ]:
# Se genera respuesta
#print(few_shot_prompt.invoke({"input":"2 ⌂ 4"}))
#print(few_shot_prompt.invoke({"input":"2 ⌂ 8"}).to_messages())

print(few_shot_prompt.invoke({}))

In [ ]:
# Se construye el prompt principal con un template de entrada para el usuario
main_prompt = ChatPromptTemplate.from_messages(
    [('system','Eres un mago de las matematicas.'),
     few_shot_prompt,
     ("human", "{input}")
    ])

In [ ]:
# Se integra el main_prompt con el modelo
chain = main_prompt | llm

# Se invoca la cadena y se pasa la entrada para que la IA interprete
#chain.invoke({"input":"2 ⌂ 9"}).content
chain.invoke({"input":"¿Cuanto es 5 ⌂ 9 ?"}).content



Ahora se va a mejorar la respuesta con templates de salida

In [ ]:
# Crear una plantilla de prompt para traducciones
prompt_template = ChatPromptTemplate.from_messages(
    [("system","Traduce lo siguiente al {language}:"),
     ("human", "{text}"
        ),
    ]
)


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

parser = StrOutputParser()
chain = prompt_template | llm | parser

In [ ]:
response = chain.invoke({'language':'Italian','text':'hello'})

In [ ]:
print(response)

In [ ]:
# Ahora otros ejemplos manteniendo el historial:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat_history = []

if not chat_history:
    chat_history.append(SystemMessage(content = "Eres un asistente personal de IA"))



In [ ]:
query = input('Haz una pregunta: ')

chat_history.append(HumanMessage(content = query))
response = llm.invoke(chat_history).content
chat_history.append(AIMessage(content=response))
print(ai_msg.content)

In [ ]:
for message in chat_history:
    print(message.content)

In [ ]:
chat_history

Ahora se va a trabajar con Runnable, OutputParser y Streaming

In [ ]:
!pip install langchain

In [ ]:
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)



In [ ]:
# Pasamos 10 al primer lambda y este lambda lo pasa al segundo
sequence.invoke(10)

In [ ]:
sequence = RunnableLambda(lambda x: x + 1) | {
    'index_1':RunnableLambda(lambda x: x * 2),
    'index_2':RunnableLambda(lambda x: x * 5),
}

In [ ]:
sequence.invoke(10)

In [ ]:
# Ahora vamo a obtener respuestas en formato Json
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None)


In [ ]:
joke_query = 'Tell me a joke'

In [ ]:
parser = JsonOutputParser()

In [ ]:
prompt = PromptTemplate(
    template='Answer the user query. \n {format_instructions} \n Query: {query}',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()}
)

In [ ]:

chain = prompt | model | parser

In [ ]:
response = chain.invoke({'query':joke_query})

In [ ]:
response

# Ahora vamos a probar el Streaming

In [ ]:
import time
for s in chain.stream({'query':joke_query}):
    print(s)
    time.sleep(0.3)

In [ ]:
chunks = []
async for chunk in model.astream(joke_query):
    chunks.append(chunk)
    print(chunk.content, end='', flush=True)
    time.sleep(0.5)

Ahora se va a crear un chatbot inteligente

1. Configuracion
2. Sistema de memoria
3. Interaccion

In [ ]:
# Para almacenar la información historica de la conversacion
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.runnables import RunnableWithMessageHistory

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(
  model,
  get_session_history
 )

In [ ]:
config = {'configurable':{'session_id':'abc2'}}

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='Hola, soy William')],
  config=config
)

In [ ]:
response.content

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='¿Cuál es mi nombre?')],
  config=config
)

In [ ]:
response.content

In [ ]:
store

In [ ]:
#Ahora cambiamos de ID
config = {'configurable':{'session_id':'abc3'}}


In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='¿Cuál es mi nombre?')],
  config=config
)

In [ ]:
response

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='Hola, soy Alejandro')],
  config=config
)

In [ ]:
response

In [ ]:
# Vuelvo a preguntar quien soy
response = with_message_history.invoke(
  [HumanMessage(content='¿Cuál es mi nombre?')],
  config=config
)

In [ ]:
# Se pinta la respuesta y ahora si me dice quien es el que pregunta
response.content

In [ ]:
# En el almacen estan las dos conversaciones con todos los usuarios
store

Ahora le vamos a dar personalidad mediante un prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
  [
    (
        'system','Eres un asistente util. Responda todas las preguntas con la mejor habilidad'
    ),
    MessagesPlaceholder(variable_name='messages')
  ]
)

In [ ]:
chain = prompt | model

In [ ]:
response = chain.invoke(
  [HumanMessage(content='Hola, soy William')],
  config=config
)
response.content

In [ ]:
response = chain.invoke(
  [HumanMessage(content='Cuál es tu nombre?')],
  config=config
)
response.content

In [ ]:
with_message_history = RunnableWithMessageHistory(
  chain,
  get_session_history
 )

In [ ]:
config = {'configurable':{'session_id':'abc5'}}

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='¿Cual es tu nombre?')],
  config=config
)
response.content

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='Mi nombre es Bob')],
  config=config
)
response.content

In [ ]:
response = with_message_history.invoke(
  [HumanMessage(content='¿Cuál es mi nombre?')],
  config=config
)
response.content